In [ ]:
from flask import Flask,redirect,jsonify,render_template
from flask.ext.cors import CORS
app = Flask(__name__)
CORS(app)

# @app.route("/")
# def main():
# #     return render_template('index.html')
#     return "<a href='./news/111'>go</a>"
    
# @app.route("/news/")
# @app.route("/news/<id>")
# def news(id=None):
#     print(id)
#     if id is None:
# #         return "<a href='./news'>go</a>"
#         return redirect("/")
#     return "Hello"+id

@app.route("/api/count")
def count():
    cur.execute('SELECT date,count(timestamp) FROM cnynes GROUP BY date ORDER BY date DESC')
    data = []
    for x in cur.fetchall():
        temp = {}
        temp["date"] = x[0]
        temp["count"] = x[1]
        cur.execute('SELECT pic FROM cnynes WHERE pic is not null and date == "'+temp["date"]+'" LIMIT 1')
        pic = cur.fetchone()
        if pic is not None:
            temp["pic"] = pic[0]
        else:
            temp["pic"] = None
        
        data.append(temp)
    return jsonify(data)

@app.route("/api/date/<date>")
def read(date):
    print(date)
    cur.execute('SELECT *FROM cnynes WHERE date="'+date+'" ORDER BY timestamp DESC')
    data = []
    for x in cur.fetchall():
        print(x)
        temp = {}
#         temp["id"] = x[0]
#         temp["nid"] = x[1]
        temp["date"] = x[2]
        temp["timestamp"] = x[3]
        temp["url"] = x[4]
        temp["title"] = x[5]
        temp["content"] = x[6]
        temp["keyword"] = x[7]
#         temp["pic"] = x[8]
        
        data.append(temp)
    return jsonify(data)


if __name__ == "__main__":
    app.run(host='0.0.0.0')

In [10]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

NameError: name 'app' is not defined

## 連接和新建資料庫

In [3]:
import sqlite3

def create_table(conn, create_table_sql):
    global cur
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        cur = conn.cursor()
        cur.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)
        
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None


db = "cnynes.sqlite"
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS cnynes (
                                        id integer PRIMARY KEY,
                                        nid integer NOT NULL,
                                        date text,
                                        timestamp text,
                                        url text,
                                        title text,
                                        content text,
                                        keyword text,
                                        pic text
                                    ); """

conn = create_connection(db)
if conn is not None:
    create_table(conn, sql_create_projects_table)
else:
    print("Error! cannot create the database connection.")

cur.execute('SELECT max(timestamp) FROM cnynes')
print("DB最後的新聞：",cur.fetchall()[0][0])

cur.execute('SELECT count(timestamp) FROM cnynes')
print("DB新聞比數：",cur.fetchall()[0][0])

cur.execute('SELECT count(timestamp) FROM cnynes WHERE keyword is null')
print("DB未萃取比數：",cur.fetchall()[0][0])

cur.execute('SELECT date,count(timestamp) FROM cnynes GROUP BY date ORDER BY date DESC')
print("每一天比數：",cur.fetchall())

# cur.execute('SELECT * FROM cnynes')
# print(cur.fetchall())

DB最後的新聞： 1515413482
DB新聞比數： 780
DB未萃取比數： 780
每一天比數： [('2018-01-08', 27), ('2018-01-05', 22), ('2018-01-04', 32), ('2018-01-03', 31), ('2018-01-02', 31), ('2017-12-29', 32), ('2017-12-28', 33), ('2017-12-27', 30), ('2017-12-26', 26), ('2017-12-25', 27), ('2017-12-22', 31), ('2017-12-21', 29), ('2017-12-20', 31), ('2017-12-19', 34), ('2017-12-18', 31), ('2017-12-17', 1), ('2017-12-15', 29), ('2017-12-14', 28), ('2017-12-13', 32), ('2017-12-12', 35), ('2017-12-11', 30), ('2017-12-10', 1), ('2017-12-08', 28), ('2017-12-07', 32), ('2017-12-06', 35), ('2017-12-05', 32), ('2017-12-04', 36), ('2017-12-03', 1), ('2017-12-01', 13)]


## 爬未載入db的新聞鏈接和標題

In [5]:
import requests
import urllib.request 
# from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import json
import datetime

base_url = "https://news.cnyes.com"
url = base_url +"/api/v3/news/category/tw_quo?startAt=1512086400&endAt=1516406400&limit=30&page=1"

cur.execute('SELECT max(timestamp) FROM cnynes')
max_time = cur.fetchall()[0][0]
if max_time is None:
    max_time = 0
else:
    max_time = int(max_time)
print(max_time)
news = []
continueToUrl = True
while(True):
    print(url)
    r = requests.get(url)
    data = r.json()
    for item in data["items"]["data"]:
        new = {}
        new["timestamp"] = item["publishAt"]
        new["date"] = datetime.datetime.fromtimestamp(int(item["publishAt"])).strftime('%Y-%m-%d')
        new["nid"] = int(item["newsId"])
        new["url"] = "https://news.cnyes.com/news/id/"+str(item["newsId"])
        new["title"] = item["title"]
        
        if max_time < new["timestamp"]:
            print(new)
            news.append(new)
            colvalue = (new["timestamp"],new["date"],new["nid"],new["url"],new["title"])
            cur.execute("INSERT INTO cnynes('timestamp','date','nid','url','title') VALUES (?,?,?,?,?)",colvalue) 
            conn.commit()
            print(cur.fetchall())
        else:
            continueToUrl = False
            break
#         break
    
    if continueToUrl is True:
        if data["items"]["next_page_url"] is not None:
            url = base_url+data["items"]["next_page_url"]
            print("Continue to next page")
#         break
    else:
        break

1515413482
https://news.cnyes.com/api/v3/news/category/tw_quo?startAt=1512086400&endAt=1516406400&limit=30&page=1
{'timestamp': 1515577896, 'date': '2018-01-10', 'nid': 4012927, 'url': 'https://news.cnyes.com/news/id/4012927', 'title': '【元大期貨】台股期指盤後－電子權值領跌，台指期拉回！'}
[]
{'timestamp': 1515576665, 'date': '2018-01-10', 'nid': 4012880, 'url': 'https://news.cnyes.com/news/id/4012880', 'title': '【華冠投顧】非電子領軍向錢衝 補教新貴華麗登場'}
[]
{'timestamp': 1515576177, 'date': '2018-01-10', 'nid': 4012879, 'url': 'https://news.cnyes.com/news/id/4012879', 'title': '【華冠投顧】劉彥良 盤後分析'}
[]
{'timestamp': 1515575653, 'date': '2018-01-10', 'nid': 4012867, 'url': 'https://news.cnyes.com/news/id/4012867', 'title': '【華冠投顧】武傑電台: 回測10日線!'}
[]
{'timestamp': 1515575559, 'date': '2018-01-10', 'nid': 4012848, 'url': 'https://news.cnyes.com/news/id/4012848', 'title': '【豐銀投顧】無懼大盤回檔 信昌化逆勢漲停'}
[]
{'timestamp': 1515574075, 'date': '2018-01-10', 'nid': 4012729, 'url': 'https://news.cnyes.com/news/id/4012729', 'title': '【華冠投顧】指數長黑拉回後、十日

## 爬未載入db的新聞內容

In [8]:
cur.execute('SELECT id,url FROM cnynes WHERE content is null')
for id,url in cur.fetchall():
    print(id,url)
    r = requests.get(url)
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.content,"lxml")
    news_text = soup.select("div[itemprop:articlebody]")[1].text
    print(news_text)
    cur.execute("UPDATE cnynes SET content = ? WHERE id = ?",(news_text,id)) 
    conn.commit()  
    
    pic = soup.find("article").find('img')
    if pic is None:
        pic = None
    else:
        pic = pic.attrs['src']
        print(pic)
        cur.execute("UPDATE cnynes SET pic = ? WHERE id = ?",(pic,id)) 
        conn.commit()   
#     break

583 https://news.cnyes.com/news/id/3987191
◆盤勢分析
大盤站上五日線，10500 整數心理關卡得而復失，量縮紅 K 做收！
台股今日依然在半年線下震盪，呈現開高走低的格局，正如同昨日所說，上檔面臨重重的壓力（十日、月線、季線及半年線壓力）以及近四個多月來，10500 以上的套牢賣壓區，後續因上檔的壓力較為沈重，因此短線上要再上攻的困難性亦較高～另近期指數當日震盪較為劇烈，今日的震幅亦有 66 點之多。收了一根帶上影線的小紅 K，行情弱時有漲的個股、行情強時亦有跌的個股，未來應留意個股的多、空趨勢來做選股，而非隨大盤的起落而追高殺低，另操作上無論作多或作空，皆應嚴設防守點，以免震盪劇烈的行情當中蒙受大賠的遺憾！
◆ 今日盤面焦點:
台股今日類股呈現漲多於跌，其中漲幅最高是其他電子類股，達 2.3%，光電漲幅也超過 2%，玻陶、電腦設備、生醫、電器電纜、電機機械、其他類等漲幅在 1% 以上。
至於下跌股則有汽車、塑膠、食品等。
鴻海今日成多方強勢指標，早盤開盤就跳空越過 5 日線，帶動台積電及大立光跟進反彈，另外鴻家軍也跟進，京鼎亮燈漲停，中揚光、廣宇、訊芯 - KY、正達、群創、台揚、榮創、GIS-KY、乙盛 - KY、臻鼎 - KY 等漲幅也超過 1%。
中小型股則由被動元件、MOSFET、矽晶圓等電子次族群走揚，其中被動元件股由同屬國巨集團的凱美亮燈漲停，立隆電、華容、奇力新、立敦、奧斯特、智寶、華新科、越峰、美磊、光頡、禾伸堂、九豪等也均上漲逾 1%。

584 https://news.cnyes.com/news/id/3987173
陳武傑分析師
◆ 收復半年線
低檔買盤持續擴散，早盤加權指數上漲 + 39 點開出，在 2317 鴻海的強彈下，指數一路震盪走高；收盤雖然受到台積電的摜壓，終場依舊上漲 + 74 點作收，收在 10473 點，成交量 1167 億。
延續上星期五的分析… 買盤開始擴散，多方預計將主導本周盤勢，預計指數將先『收復半年線』。今天指數初步先來挑戰 10500 點，雖然尾盤無法成功站上；但是就今天籌碼流動看… 在外資持續減碼 - 37 億下，指數依舊可以收漲 + 74 點，顯示黑手低檔買盤持續加溫。再加上今天成交量僅 1167 億，表示指數多方攻擊並無壓力，投資朋友保持多方操作；指數本周還是

台股今 (11) 日由鴻海擔任攻堅主力，開盤順利收復 5 日線，蘋概指標台積電、大立光及可成皆同步上漲，電子次族群被動元件、MOSFET、LED 也強勢反彈，推升指數一度大漲 106 點，站上 10500 點整數關卡，不過尾盤漲幅收斂，終場上漲 74.47 點，以 10473.09 點作收，成交值 1164.45 億元。
櫃買市場漲勢更較集中市場猛烈，指數開高走高，終場大漲 2.01 點，以 146.42 點作收，成交值 436.28 億元。
台股今日類股呈現漲多於跌，其中漲幅最高是其他電子類股，達 2.3%，光電漲幅也超過 2%，玻陶、電腦設備、生醫、電器電纜、電機機械、其他類等漲幅在 1% 以上。至於下跌股則有汽車、塑膠、食品等。
鴻海今日成多方強勢指標，早盤開盤就跳空越過 5 日線，帶動台積電及大立光跟進反彈，指數也順利站上 10500 點，另外鴻家軍也跟進，京鼎亮燈漲停，中揚光、正達、台揚、榮創、群創、GIS-KY、臻鼎 - KY、乙盛 - KY、廣宇、訊芯 - KY 等漲幅也超過 1%。
中小型股則由被動元件、MOSFET、矽晶圓等電子次族群走揚，其中被動元件股由同屬國巨集團的凱美亮燈漲停，立隆電、立敦、奧斯特、智寶、華新科、禾伸堂、華容、奇力新、九豪、光頡、越峰、美磊等也均上漲逾 1%。
MOSFET 族群則由尼克森、富鼎亮燈漲停，大中、茂達等也均收紅。矽晶圓龍頭廠環球晶雖然早盤一度震盪，不過中場漲幅開始放大，帶動合晶、中美晶、嘉晶等收紅。
另外功率放大器族群漲勢也相當一致，穩懋因為攜手 Avago 亮燈漲停，環宇 - KY 也同步亮燈，全新、IET-KY、宏捷科等漲幅也超過 5%。
分析師林孟漢表示，台股今天主要還是靠蘋概股強攻，其中鴻海更是強勢指標，一吐之前被外資狂賣的怨氣，也帶動台積電及大立光等權值股止穩，至於中小型股表現也不錯，拉抬指數一度收復 10500 點，不過整體來看，11 月營收表現仍是短線主要影響因素，預料下半周開始，營收因素將減弱，將考驗台股漲勢後續動能的關鍵。

https://cimg.cnyes.cool/prod/news/3986879/l/fb80d7a306462542ddd6f841e6ac3759.jpg
594 https://news.cnyes.com/news/id/3986554
台股今 (11) 日由鴻海

◆盤勢分析
昨天提到：「在大盤續創新低 10322 之時，已經回補 9/30 向上缺口，加上月線負乖離過大，後續將轉為震盪或反彈」。今天大盤如預期反彈收小紅，更重要的是原本波段走勢就強於大盤的 OTC 指數今天小破半年線後收腳，終場大漲 1.65%，K 線呈現「多頭吞噬」型態，暗示內資認為大盤再跌有限，作多中小型股的心態轉強，反映在盤面就是 "強勢股領先邁步走"。
週三提到：「強勢主流 3D 感測族群 (3105 穩懋、2455 全新、8086 宏捷科、3227 原相等) 屬於創高後拉回，看起來大人還在，短線應會伺機再拉高一次」，今天果然群起而上。另一個強勢主流被動元件，今天 2327 國巨、2492 華新科、3026 禾伸堂、6173 信昌電也都聯袂大漲。原強勢主流猛攻就是 "強勢股領先邁步走" 的最佳寫照。
先前持續鎖定的設備股 (3583 辛耘、3563 牧德、6208 日揚、5536 聖暉) 近期表現也都強於大盤，尤其是 3583 辛耘在法人沿路加碼、融資持續減少下，呈現強勢整理態勢，今天一馬當先鎖上漲停，也是典型的 "強勢股領先邁步走"。相信其他個股表現不會寂寞。
過去 2 周外資持續倒貨把台股搞得淒風苦雨，昇楷很清楚好朋友心中的苦，當然也盡力想幫大家，歡迎加入 Line 群組，本周末會直接分享看好的太極強股，未來一起成為快樂的贏家。
※ 以上內容由華冠投顧 賴昇楷 分析師 整理提供，投資人於投資前請審慎評估，遵守紀律，嚴設停利停損。
歡迎免費加入 賴昇楷 老師的 Line 群組 ID： https://goo.gl/F7NFC6
服務電話：(02)6630-3221
華冠投顧官網：http://www.hwa-guan.com.tw
 

605 https://news.cnyes.com/news/id/3985738
◆盤勢分析
前言：台股盤中再殺一波後出現拉抬，大盤日 K 留下影線，OTC 的紅 K 吞噬昨日的黑 K
台股這三天的融資餘額下降，上市櫃合計減了約 42 億，但降得不夠，籌碼還不夠乾淨，所以盤這兩天就算開高，盤中還是要殺盤，洗乾淨了才能拉抬。大盤收盤漲 42 點（0.41%），日 K 留下影線；OTC 更強，破底後拉出紅 K，直接吞噬昨日黑 K，漲幅達 1.65%！
類股分析：上市漲幅前五名為玻璃陶瓷、電機機械、造紙、電器電纜與電子零

◆盤勢分析
期貨走勢
台指期週五上漲 31 點至 10396 點。價差方面，台指期轉為逆價差 - 2.62 點，電子期轉為逆價差 - 0.3 點，金融期轉為逆價差 - 0.37 點。現貨部分，三大法人賣超 21.48 億元；而在台指期淨部位方面，三大法人淨空單增加 1229 口，使留倉部位轉為淨空單 - 102 口，其中外資多單減碼且空單加碼，淨多單減少 2604 口至 36618 口；十大交易人中的特定法人全月份台指期淨多單減少 1614 口至 6661 口。
週四美股三大指數同步上漲，亞股早盤普遍開高，帶動今日台股則以上漲 50 點開出，先前跌深的權值股台積電，大立光出現反彈，但指數反彈一度遭遇賣壓由紅翻黑，所幸 10 點過後低接買盤進場拉抬指數，午盤過後指數進一步上揚，終場加權指數上漲 42.86 點或 0.414%，收在 10398.62 點，成交量能 1299.49 億，較前一日縮減 85.87 億。外資今日在台股現貨持續賣超，但金額已見縮減，同時減碼台指期淨多單，顯示外資心態持續偏向保守看待。不過櫃買市場今日則出現較明顯反彈，收盤上漲 1.65% 以帶下影線紅 K 吞噬前日黑 K 棒，止跌訊號初現。技術面觀察，週五大盤開高走低以帶下影線黑 K 反彈上揚，距 10400 僅一步之遙，短線止跌跡象浮現，惟上方半年線位置反壓仍重，且短期均線呈空頭排列，需待整理過後多方有向上挑戰機會。
選擇權分析
 
選擇權從成交量來觀察，買權集中在 10500 點，賣權集中在 10300 點。選擇權未平倉量部分，買權未平倉最大量集中在 10900 點，賣權未平倉最大量集中在 10000 點。全月份未平倉量 put/call ratio 值維持 1.26，大於中性值 1，反應籌碼面為偏多架構。買權平均隱含波動率由 13.21% 降至 11.92%，賣權平均隱含波動率由 14.38% 降至 13.29%，買賣權皆降。選擇權部位整體來看顯示仍為中性偏多預期。
本投資研究報告由永豐期貨股份有限公司版權所有，未經授權，禁止轉載節錄。本資料內容僅供參考，客戶應審慎考量本身之需求與投資風險，本公司恕不負任何法律責任，亦不作任何保證。

616 https://news.cnyes.com/news/id/3984895
◆ 劉彥良 盤後分析
昨天一早在 Line 生活圈就告訴投資朋友，加權


◆盤勢分析
美科技股跌深反彈，激勵 7 日台股開高，股價王侵權官司勝訴領漲大盤，惟市值王盤中疲弱引發賣壓，金融及傳產亦無力，使得台股連三日收低，終場加權指數收在 10355.76 點，下跌 38.16 點或 0.37%，成交額 1385.36 億元。 OTC 指數收在 142.07 點，下跌 1.31 點或 0.91%，成交額 423.79 億元。
7 日台股再遭空方打擊，僅管多方盤中力圖振作，但 11 月營收衰退個股，股價遭摜壓近跌停，加上追價買盤不足，令集中及櫃買指數同步收黑，技術面弱入超賣區。籌碼方面，三大法人連六日同步賣超，其中外資連賣十天，期現貨維持偏空操作。由於週四低點未破前低 (9/26 10257 點)，加上日線留長下影線，且技術面來到超賣區，預期下檔空間有限，未來觀察公司 11 月營收表現、費城半導體指數走勢、蘋果走勢及法人操作等再研判台股方向。
 

627 https://news.cnyes.com/news/id/3983946

◆強攻失利，台股回跌
週四台股開高震盪走跌，加權指數早盤急漲站上 10400 點，隨後轉弱翻黑，收盤下跌逾 38 點。終場台指期下跌 24 點 (-0.23%)，正價差 4.24 點，電子期下跌 0.05 點 (-0.01%)，正價差 0.27 點，金融期下跌 4.4 點 (-0.37%)，正價差 2.93 點，非金電期下跌 16 點 (-0.13%)，正價差 17.94 點。台指期下跌，逆價差轉為正價差，期貨多方力道增強。
台指期未平倉部位分析，自營商淨空單增加 700 口，淨空單部位 4215 口；投信淨空單減少 573 口，淨空單部位 33880 口；外資淨多單減少 928 口，淨多單部位 39222 口，外資持有期貨作多部位。
技術面觀察，加權指數日線收黑 K 棒，各短中期均線呈現反轉下彎，籌碼面部分，單日成交量能約 1375 億餘元，外資上市現貨賣超金額約 67.56 億元，期貨未平倉淨多單 39222 口，外資現貨多單、期貨多單同步減碼。
◆選擇權分析
多空就選擇權籌碼觀察，賣權 / 買權未平倉比由 1.32 變動到 1.26，為偏多格局。在最大未平倉量序列部份，買權維持在 10,900 點，賣權維持在 10,400 點，外資選擇權淨空單部位 238,574 口。
 

628 https://ne

...

陳武傑分析師
◆ 10500-10800
台北股市早盤上漲 + 31 點開出，但是在大立光的領跌下，指數由紅翻黑；隨後在金融股買盤推升下，指數衝高一度大漲 + 105 點，收盤上漲 + 39 點作收，收在 10600 點，成交量 1791 億。
延續我們昨天的分析… 短線觀盤重點依舊在『融券』，只要券單可以順利? 加；年前的紅包行情隨時都會展開。但是昨天『融券』意外不增反減，早盤 10：09 在 LINE@已經跟大家說：『昨天『融券』不增反減… 是今天彈升無力的主因。恐荒性賣盤出來了… 這是加速趕底…』；終場三大法人同步賣超台股 - 114 億，顯示黑手『宣示主權』意味濃厚。展望下星期，雖然金融股領軍，但是多方目前『只守不攻』；預計短期先在 10500-10800 區間震盪，等待賣壓消化後再出發。
今天台股上沖下洗開盤漲 31 點，早盤在大部份投資人都認為要開始反彈之際，突然下殺翻黑至 65 點，到大約 11 點後又翻紅一路上揚，最多漲幅來到 105 點，但隨後又有拉回終場只僅僅收漲 39 點，收在 10600 點，從十月中旬開始指數就在 10800 點附近上上下下，一直持續到 11 月初有個比較像樣的拉回修正，11 月 16 日開始武傑幾乎天天給投資人信心，從『盤漲』、『再創新高』、『量價齊揚』、『創高不是結束』… 等，11 月 22 日指數再創新高後這幾天有比較快速的拉回修正，，但武傑對今年的看法依舊沒變，大跌了又是一次佈局的機會點，只是整理振盪期恐會拉長，所以大家切勿過度恐慌，有拉回修正時是給大家機會低檔布局的，大家要好好把握，依舊給大家信心，而選股方面依舊建議往有基本面的中小型業績成長股，為投資的首選，最後再次提醒大家目前整體今年多頭走勢並沒改變，記得逢低佈局! 逢高獲利! 不要隨市場消息起舞追高殺低! 喜歡武傑分析的朋友都可以在 LINE@或武財神陳武傑 FB 上有更多即時資訊與你分享!
 
個股方面今天股王 3008 大立光下跌收在 5010 元， 2317 鴻海下跌收在 99.1 元，權值王 2330 台積電則是上漲 2.22% 收 231 元 (占了指數 43 點)，再來幫大家追蹤今天盤面上強勢的個股有: 4943 康控 KY、4916 事欣科、8163 達方、4979 華星光、2486 一詮、8171 天宇、3508 位速、1528 恩德… 等，有

◆盤勢分析
1. 美股維持強勢，週三台股早盤再創 27 年來新高，最高來到 10934.75 點，但受到電子股走弱影響，終場指數下跌 83.80 點，跌破五日均線，來到 10831.09 點，成交量為 1375.61 億元。
2. 觀察台股技術面，在法人大買小賣推升下，帶動近期台股屢創波段高點，不過短線上與月線乖離達 3% 左右，已出現過熱現象，週三賣壓湧現，順勢回檔，雖跌破五日均線，但目前短、中、長期均線持續揚升，加上國際盤仍偏多，目前以漲多回檔看待，後續關注法人持股狀態以及技術面多方缺口 10710 點是否在回檔過程中回補，即可，當前先視為漲多修正，後市並不看淡。
3. 三大法人合計賣超 27.68 億元，外資與陸資 - 32.21 億元、投信 + 3.35 億元、自營商 + 1.18 億元。
非電仍是主軸  金融轉強  電子結構仍在調整，留意矽晶圓、玻纖布、電競等強勢族群
1. 在電子股指數走勢相對偏弱，季線未能站上，且跌破十日均線，且高檔五日均線出現下彎，強勢權值股台積電亦湧現賣壓，而 12 月台積電營收 898.97 億元，為近三個月新低，惟略優於法說預期，觀察台積電營收公布後法人動態。弱勢包括 3008 大立光、2317 鴻海、2308 台達電等，11 日 (週三) 大立光將召開法說，可視為電子股重要指標，若大立光能回穩，對電子股短線急殺後，或有轉強契機。族群方面，目前以具有價格敏感性的像是玻纖布、矽晶圓等股價位階較強，像是 12 月營收持續成長的 5475 德宏、5340 建榮、5481 華韡等，矽晶圓 6488 環球晶、6182 合晶 12 月營收皆創歷史新高，股價短線回檔，技術面仍偏多，3532 台勝科股價續強；電競市場持續成長，亦帶動 2353 宏碁相對強勢、2376 技嘉走強，二極體受惠漲價題材，2481 強茂、5305 敦南等吸引法人增持，IC 設計 2454 聯發科推出 AI 平台、6202 盛群受惠 MCU 漲價、4947 昂寶 - KY 打入聯想供應鏈、5274 信驊受惠新伺服器平台帶動業績。股價走強。
2. 非金電，近期為盤面的主軸，NYMEX 原油再創 3 年新高，塑化股交投維持熱絡，1301 台塑、1326 台化盤中再創波段新高，1303 南亞受惠玻纖布漲價，股價維持強勢。SM 價差維持穩定，帶動 1312 國喬、1310 台苯強


◆盤勢分析
昨日美股漲跌互見，除 S&P 500 與 Nasdaq 再創高外，道瓊也重回漲勢續創新高，惟費半走跌 0.97%。本周起迎來美國超級財報周，在市場預期減稅有利財報表現下，樂觀看待並使美股漲勢延續，其中部分獲利良好企業已開始公布訊息，如零售商 Target 公布去年假日季節銷售成長 3.4% 高於預期，昨日股價上漲 2.9%。整體來說在景氣持續成長、市場樂觀看待基本面下，研判美股將可維持目前多頭趨勢不變，四大指數僅需留意短線上漲多後可能出現的拉回修正走勢即可，在未跌破月線前，持續偏多看待。
台股昨日開盤再創波段新高，但隨即遇獲利賣壓使指數翻黑，所幸盤中跌幅收斂，終場收在 10914 點，成交量為 1299 億。指數終止連八紅，台股居高思危，量能進一步萎縮，顯示市場風險意識仍高，在指數來到高檔之際追價意願下滑，台股連四日出現吊人線，加上乖離仍大，研判短線走勢將維持震盪，但只要跳空缺口 10710 點未遭回補，多頭就沒有轉弱的疑慮，待指數震盪、乖離縮小後，仍有機會向上挑戰 11000 點大關。
觀察盤面，昨日傳產類股以塑化及造紙族群表現較佳，電子類股多為個股表現。塑化族群在油價維持高檔下，台塑、台化等漲幅皆超過 3%，此外塑化產品受惠油價高漲推動報價提升，上週 EG 報價勁揚 7%、SM 報價亦來到去年第四季以來的新高，相關生產廠包括台苯、國喬、中纖、南亞等也有表現，在伊朗動盪加上 OPEC 減產效益延續下，有助油價維持高檔，並帶動塑化族群營運增溫。造紙族群受惠紙漿售價調漲，2017 年北美長纖紙漿價格上漲 41%，在外傳價格將再調漲下，華紙上漲逾 4%，帶動正隆、永豐餘等造紙股收紅，中國供給側改革淘汰過剩產能，紙類供需改善，後續有望在攻。個股部分如雷虎 12 月營收創當年新高，達興材料受惠中國 8.5 代線以上面板量產，推升顯示器材料需求，股價也攻上漲停。整體來看，電子股漲多休息，傳產股接棒上攻，但研判電子族群僅是短暫休息，待重整旗鼓後，在漲價、展覽等題材加持下仍有機會重現攻勢，籌碼安定、股價位階偏低或是修正完成底部之標的皆是可以留意的焦點，電子相關族群可參考矽晶圓、被動元件、PCB、車用電子、電競、3D 感測等。傳產就看供需改善、原物料價漲族群，可參考塑化、鋼鐵、紡織、造紙、玻璃陶瓷等。
 

800 https://news.cnyes.com/new

◆盤勢分析
期貨走勢
台指期週二下跌 5 點至 10879 點。價差方面，台指期逆價差擴至 - 35.89 點，電子期逆價差擴至 - 1.48 點，金融期逆價差縮至 - 2.2 點。現貨部分，三大法人買超 11.58 億元；而在台指期淨部位方面，三大法人淨多單增加 2605 口至 19592 口，其中外資多單加碼超過空單加碼，淨多單增加 3639 口至 53313 口；十大交易人中的特定法人全月份台指期淨多單增加 2887 口至 10080 口。
週一美股那斯達克與標普指數續創歷史新高，且南北韓會面激勵週二亞洲股市持續走高，加權指數早盤開高走高一度再創 27 年半新高，但台幣升值匯損壓力導致電子股壓力沉重，電子指數率先翻黑下跌拖累大盤翻黑，股王大立光持續受到營收衰退利空襲擊，盤中一度重挫近 5% 並創下波段新低，且台積電與鴻海也同步壓在盤下，所幸尾盤買盤敲進台積電與壽險雙雄，推升指數跌幅收斂僅以微跌作收，終場加權指數下跌 0.86 至 10914.89 點，成交量能 1299.49 億，較前一日縮減 71.21 億。技術面來看，週二日 K 終結連八紅並一度回測 5 日線，但仍留下長下影線並連八日越過昨高，多頭結構強勁，且量價結構穩健，預期行情有望隨國際股市強勢推升指數持續創高，不過行情逼近 11000 點整數大關壓力，逢高獲利了結賣壓也將逐步加重。
選擇權分析
 
選擇權從成交量來觀察，買權集中在 10900 點，賣權集中在 10700 點。選擇權未平倉量部分，買權未平倉最大量集中在 11000 點，賣權未平倉最大量集中在 10200 點。全月份未平倉量 put/call ratio 值由 1.84 降至 1.83，大於中性值 1，反應籌碼面為偏多架構。買權平均隱含波動率由 9.27% 降至 8.7%，賣權平均隱含波動率由 14.53% 降至 13.95%，買賣權皆降。選擇權部位整體來看顯示仍為中性偏多預期。
本投資研究報告由永豐期貨股份有限公司版權所有，未經授權，禁止轉載節錄。本資料內容僅供參考，客戶應審慎考量本身之需求與投資風險，本公司恕不負任何法律責任，亦不作任何保證。

811 https://news.cnyes.com/news/id/4010781
一般的散戶總是試圖找尋獲利的捷徑，最好像能有個指標，然後有個訊號 (有人喜歡稱符號)，能讓自己輕易的對股

◆盤勢分析
美元弱，台幣強，傳產、金融股勇往直前，加上外資看好市值王持續加碼，激勵 8 日台股開高走高，無視股價王 12 月營收衰退、部分外資調降目標價利空衝擊，在類股良性輪動下，大盤再創 27 年新高，終場加權指數收在 10915.75 點，上漲 35.95 點或 0.33%，成交額 1393.02 億元。
OTC 指數收在 152.12 點，下跌 0.35 點或 0.23%，成交額 462.35 億元。
8 日台股日線連八紅，各期均線同步向上，技術面處於高檔過熱區，不過量能在五日線之上，三大法人也同步站在買方，其中外資連八日買超，累計金額 420 億。考量國際股市走勢亮麗，目前雖然技術面位於高檔，但在法人偏多操作，基本面表現樂觀下，台股有機會再創佳績，選股可留意 12 月營收優於預期、台幣升值概念股。

822 https://news.cnyes.com/news/id/4009855
 ◆盤勢分析
1. 台股昨日開高震盪，雖然早盤一度翻黑，但隨即拉高表現，尾盤再見拉升，再創波段新高表現，日線拉出連 8 紅，終場以上漲 35.95 點作收。短線上月、季線趨勢翻揚向上，並且也有多方缺口，再缺口未回補前，依舊多方格局，量能若能持續，方有機會上攻，籌碼面外資再加碼台股，挹注台股動能，持續觀察外資買盤。
2. 櫃買指數相對加權指數弱勢，開高走低，尾盤浮現調節賣壓，不過整體仍是多方回檔，後勢量價若未出現背離，中小型個股仍有機會表現。
3. 金融指數擴大漲幅，再創波段新高表現，多方格局不變，後勢可持續追蹤。電子指數仍受季線反壓影響，且部分權值股修正，因此仍需觀察權值股走勢，而強勢族群有 PCB、工業電腦、矽晶圓、電競相關等。傳產類股則由玻璃陶瓷、水泥、營建等表現較佳。
4. 去 (106) 年 12 月單月出口值為 295.1 億美元，也創歷年同期新高，且連 15 個月正成長，全年出口總值重新站上 3,000 億美元大關。
5. 科技市調機構 Canalys 指出，谷歌 (Google)Google Home、亞馬遜(Amazon)Echo 以及蘋果(Apple)HomePod(預計 2018 年初期開賣) 等智慧音箱產品將成為今年 (2018 年) 成長最快速的消費性科技產品。

823 https://news.cnyes.com/news/id/4009848
◆ 

## 萃取未有keyword的資料

In [9]:
import jieba
import jieba.analyse

def index_of_keyword(sentences, keyword):
    for i, s in enumerate(sentences):
        if keyword in s:
              return i
    return -1

cur.execute('SELECT id,content FROM cnynes WHERE keyword is null')
for id,content in cur.fetchall():
#     print(id,content)
    print(id)
    
    keywords = jieba.analyse.extract_tags(content, topK=3, withWeight=False)
    keywords2 = jieba.analyse.textrank(content, topK=3, withWeight=False) 
    keywords_all = keywords+keywords2

    s_index = []
    for keyword in keywords_all:
        sentences = content.replace("\n","").split(u"。")
        s_index.append(index_of_keyword(sentences,keyword))

    s_set = set(s_index)
    s_index = list(s_set)

    summary = ""
    for i in s_index:
        summary += sentences[i]+u"。\n"

#     print("content",content)
#     print ("summary",summary)
    
    conn.cursor().execute("UPDATE cnynes SET keyword = ? WHERE id = ?",(summary,id)) 
    conn.commit()        
#     break



Building prefix dict from the default dictionary ...


1


Dumping model to file cache C:\Users\Vay\AppData\Local\Temp\jieba.cache
Loading model cost 1.327 seconds.
Prefix dict has been built succesfully.


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

## backup

In [5]:
# cur.execute('SELECT id,url FROM cnynes WHERE pic is null')
# for id,url in cur.fetchall():
# #     print(id,url)
#     r = requests.get(url)
#     r.encoding = "utf-8"
#     soup = BeautifulSoup(r.content,"lxml")
#     pic = soup.find("article").find('img')
#     if pic is None:
#         pic = None
#     else:
#         pic = pic.attrs['src']
#         print(pic)
#         cur.execute("UPDATE cnynes SET pic = ? WHERE id = ?",(pic,id)) 
#         conn.commit()   
# #     break

In [6]:
from bs4 import BeautifulSoup as soup

news_text = soup.find("article")
# if news_text is None:
#     news_text = None
# else:
#     news_text = news_text.attrs['src']
# print(news_text)

AttributeError: 'str' object has no attribute 'find_all'

In [ ]:
.find('img')